In [ ]:
### Header
# Jared Gibbs
# 3/25/25
# Property Tax App

In [ ]:
### NOTES
"""
- Questions:
    - How does the county assessting and collecting levy get set?
    - How are district boundaries tracked?
    - 

- [59-2-1603] State auditor 'shall annually conduct a study of each county of the fourth, fifth, or sixth class to determine:
     (a) the costs of assessing and collecting property taxes;
     (b) the ability to generate revenue from an assessing and collecting levy; and
     (c) the tax burden of levying a property tax sufficient to cover the costs of assessing and collecting property taxes. 
        Can I get these reports? Talk to Seth Oveson from Auditor's Office? https://auditor.utah.gov/about-us/directory/
    
- Property Tax Policy Vairables:
    - Tax Rate (cretified rate vs marginal rate)
    - Primary residential exemption threshold
    - Curcuit Breaker?
    - Tax Credits?
    
- Functionalities:
     - Model Changes in tax rate
     - Model Changes in exemption(s)
     - Tax incidence by property type
     - Tax incidence by income?
     - Variable geographic resolution
     - Map graphics?
     - Sankey for where property taxes are levied and where they go?

- Create an index to explain each variable?

- Tax flows:
    - County/ municipal             ->  local government(s)
    - County assessment levy        ->  promote the efficient administration of the property tax system
    - Mulitcounty assessment levy   ->  18% (up to $500,000) to Property Tax Valuation Fund, rest to Multicounty Appraisal Trust 
    - School district               ->  school districts
    - Special districts             ->  special district
        
- Resources
    - Tax Area Nomenclature https://propertytax.utah.gov/rates/nomenclature/
    - Property Tax Levies Statutory Authority and Maximums https://propertytax.utah.gov/tax-rates/property-tax-levies.pdf
    - Final Countywide Taxable Values Report 233b (Form PT-200) https://propertytax.utah.gov/form/pt-200.pdf
      
    - HvPlot User Guide https://hvplot.holoviz.org/user_guide/index.html
    - HoloViz Tutorial https://holoviz.org/tutorial/index.html
    
    - OLAG property tax audit: https://olag.utleg.gov/olag-doc/2024-05_RPT.pdf

    - CSS option Referance https://www.w3schools.com/cssref/index.php
        
        
        
"""

In [ ]:
### To Do list 
"""
General
    Dynamic map wrapper?
    District Map
    
    Count Timeseries
    Average TimeSeries
    Average profiles
    
    Discussion of Truth in Taxation
        How does it work?
        Tooele City as impirical example of TnT in action
        Implications for the taxpayer
        Why are my taxes going up?

    Modeling Page
    Reorganize Statewide comparison page
    Reorganize entity profile page



Double Checking
    review ratio calcualtions for early years
    cross check app results with taxing entitiy budget information
    review personal property revenue calcuations

    

Modeling
    Decompose value growth due to increase in stock vs increase in value
        dicussion of truth in taxation implications for individual taxpayers 
    Policy Variables:
        Statewide marginal property tax change
        Change to the primary residential exemption



Additional Research
    Curcuit Breaker
    Existing Targeted property tax breaks

"""

In [ ]:
### Libraries and Imports

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import datashader, hvplot, param, holoviews as hv
import geoviews as gv
import panel as pn
import colorcet as cc
from colorcet.plotting import swatch, swatches
hv.extension('bokeh', 'matplotlib')
import hvplot.pandas
import pathlib

df_cleandata = pd.read_csv('property_cleandata.csv')
#df_cleandata_county = pd.read_csv('property_cleandata_county.csv')

df_working = pn.rx(df_cleandata)
#df_working_county = df_cleandata_county

#Set up colors and colormaping
highlight = '#196B24'
alt_highlight ='#8b4aa8'

base_colors = ['#196B24', '#8b4aa8', '#767167', '#bcb4a5', '#171615',  '#2f2d29', '#47443e', '#5e5a52', '#8d877c', '#a59e90', '#d4cbb9', '#ebe1ce']
swatch(name= 'base_colors', cmap = base_colors)



In [ ]:

blog_post_data = df_cleandata[['Tax Year', 'Total Property', 'entity_code', 'entity_name']]
blog_post_data = blog_post_data[(blog_post_data['entity_code']>=5000) & (blog_post_data['entity_code']<6000)]

blog_post_data.to_excel('blog_post_data.xlsx', index=False)

In [ ]:
### Set Up Widgets et al

entites = df_cleandata['entity_name'].unique().tolist()
entity_types = df_cleandata["entity_type"].unique().tolist()
counties = df_cleandata['county_name'].unique().tolist()
#counties.remove("Statewide")

property_types = ['Primary Residential', 'Non-primary Residential', 'Commercial', 'Agricultural', 'Unimproved', 
                  'Personal Property', 
                  'Centrally Assessed']

property_share_types = ['Primary Residential, Percent', 'Non-primary Residential, Percent', 'Commercial, Percent', 'Agricultural, Percent', 'Unimproved, Percent',
                  'Personal Property, Percent', 
                  'Centrally Assessed, Percent']

property_average_types = ['Primary Residential, average', 'Non-primary Residential, average', 'Commercial, average', 'Agricultural, average', 'Unimproved, average', 
                  'Personal Property, average']




entitiy_widget = pn.widgets.AutocompleteInput(name = "Entitiy", options = entites, case_sensitive = False, value = 'Beaver')
entity_type_widget = pn.widgets.MultiSelect(name= 'Taxing Entity Type', options= entity_types, value = ['County'])
county_widget = pn.widgets.MultiSelect(name = 'County', options = counties, value = counties)
tax_year_widget = pn.widgets.EditableIntSlider(name = 'Tax Year', fixed_start= 2014, fixed_end = 2024, step = 1, value = 2024 )
share_vs_values_widget = pn.widgets.Select(name = 'Display Type', options=['Share', 'Values'], value='Share')
sort_share_widget = pn.widgets.Select(name = 'Ratio Sort by', options = property_share_types)
sort_value_widget = pn.widgets.Select(name = 'Value Sort by', options = property_types)
ascending_widget = pn.widgets.Switch(name= 'Ascending?')


# Page Buttons
frontpage_button = pn.widgets.Button(name = "Home Page", 
                                     icon = 'home', 
                                     icon_size = '24px', 
                                     height = 50, 
                                     styles={"width": "100%", "border-style": "outset"})

entityprofile_button = pn.widgets.Button(name = 'Entity Profiles', 
                                         icon = 'file-info', 
                                         icon_size = '24px', 
                                         height = 50, 
                                         styles={"width": "100%", "border-style": "outset"})

statewidecomp_button = pn.widgets.Button(name = "Statewide Comparison",
                                          icon = 'layers-difference', 
                                          icon_size = '24px', 
                                          height = 50, 
                                          styles={"width": "100%", "border-style": "outset"})

modeling_button = pn.widgets.Button(name = "Modeling",
                                     icon = 'alpha',
                                       icon_size = '24px', 
                                       height = 50, 
                                       styles={"width": "100%", "border-style": "outset"})





In [ ]:
### Extract Key Variables/ Indicators

indicator_realproperty = df_working.loc[(df_working.entity_name == entitiy_widget) & (df_working['Tax Year'] == tax_year_widget), 'Total Property'].max()
indicator_realrevenue = df_working.loc[(df_working.entity_name == entitiy_widget) & (df_working['Tax Year'] == tax_year_widget), 'Revenue, Total Property'].max()
indicator_realrate = df_working.loc[(df_working.entity_name == entitiy_widget) & (df_working['Tax Year'] == tax_year_widget), 'Real Tax Rate'].max()
#indicator_


In [ ]:
### Stacked Bar Chart for Ratio and Value of Property Types by County

ratio_chart = df_working[(df_working.entity_type.isin(entity_type_widget)) & (df_working['Tax Year']==tax_year_widget) & (df_working['county_name'].isin(county_widget))].sort_values(by=sort_share_widget, ascending = ascending_widget).hvplot.barh(
                        x ="entity_name", 
                        y = property_share_types, 
                        stacked = True,
                        #rot = 60,
                        legend = False,
                        color = base_colors,
                        #frame_height = 1000,
                        #frame_width = 1400,
                        title = "Ratio of Property Types by Taxing Entity",
                        xlabel = 'Taxing Entity',
                        ylabel = 'Taxable Value, Percent',
                        responsive = True).opts(shared_axes= False)

value_chart = df_working[(df_working.entity_type.isin(entity_type_widget)) & (df_working['Tax Year']==tax_year_widget) & (df_working['county_name'].isin(county_widget))].sort_values(by=sort_value_widget, ascending = ascending_widget).hvplot.barh(
                        x ="entity_name", 
                        y = property_types, 
                        stacked = True,
                        #rot = 60,
                        legend = False,
                        ylabel = 'Taxable Value (B)',
                        xlabel = 'Taxing Entity',
                        color = base_colors,
                        #frame_height = 1000,
                        #frame_width = 1400,
                        title = "Value of Property Types by Taxing Entity",
                        responsive = True).opts(shared_axes= False)



In [ ]:
### Property Tax, a Primer

# INSERT HEADER VISUALS
# Write a discussion of 'Truth in Taxation' 

real_property_primer = pn.Card(
    pn.pane.Markdown(""" 
        Real Property refers to the land itself and improvements upon that land (59-2-102 (18), (32)). 
        In this context, 'improvements' refer generally to buildings or structures attached to the land in a way which is permanent over the useful life of the property.
        The manner in which real property is taxed depends on the particular type of property at issue, but generally it is levied as a percentage rate on the fair market value of the property.
        See the categories below for more detail. 
            """,styles = {'font-size': '16px'}),
    pn.layout.Accordion(
    #General Description of Real Property
          # Land and improvements to the land as defined in 59-2-102 (18), (32)
    ('Primary Residential', pn.pane.Markdown(""" 
                Primary residential property includes things like single family homes, duplexes, or other kinds of owner-occupied housing.
                It is defined in code as "any property used for residential purposes as a primary residence" (59-2-102(34)).
                The state constitution, among other property tax exemptions, carves out an exemption for up to 45% of the value of residential property (art. XIII, §3).
                At present, statute sets this exemption at the maximum allowed (59-2-103(3)), meaning that individuals pay taxes on 55% of their property's market value, rather than on the full value. 
            """,styles = {'font-size': '14px'})),
    ('Non-primary Residential', pn.pane.Markdown(""" 
                Non-primary residential property includes all property classified as residential but not used as a primary residence.
                This may include second homes, vacation properties, rental buildings etc.
                Current statute excludes non-primary residential property from receiving the 45% exemption (59-2-103(6)) and so is taxed at 100% of market value.
            """,styles = {'font-size': '14px'})),
    ('Commercial', pn.pane.Markdown(""" 
                Commercial property are those lands and improvements which are used for industrial or commercial purposes.
                Things like factories, office buildings, or department stores are all classified as commercial property for the purposes of taxation and are taxed at 100% of market value.
            """,styles = {'font-size': '14px'})),
    ('Agricultural', pn.pane.Markdown(""" 
                Agricultural or FAA (Farmland Assessment Act) property, in contrast to other types of real property, is assessed based on its value for agricultural production rather than its market value (59-2-503).
                This may include land used to grow useful plants or raise livestock as defined in code.  
            """,styles = {'font-size': '14px'})),
    ('Unimproved', pn.pane.Markdown(""" 
                Unimproved or vacant property is land which has not been built upon or utilized. It is taxed at 100% of market value. 
            """,styles = {'font-size': '14px'}))
    ),
                              title = 'Real Property',
                              collapsed = True,
                              sizing_mode = 'stretch_width')

personal_property_primer = pn.Card(
    pn.pane.Markdown(""" 
        Personal Property is a broad category which includes everything from passenger cars to business equipment to mobile homes. 
        While state code sets out the few explicit classifications (which are described below), it delegates rulemaking authority to the Tax Commission to create and maintain classifications for all other types of personal property (59-2-107).
        Due to the complexity, the intricacies of these classifications are not described here and can instead be found in the Commission's valuation guidelines: https://propertytax.utah.gov/personal/valuation-guidelines/
            """,styles = {'font-size': '16px'}),
    pn.layout.Accordion(
    #General Description of Personal Property
          # taxable property that is not real or centrally assessed
    ('Age-based Property', pn.pane.Markdown(""" 
                Property subject to the age-based fee includes most vehicles such as passenger vehicles, motorcycles, vessels (under 31 ft), trailers, etc. (59-2-405). 
                Rather than being assessed a percentage of the market value, age-based fees— as the name implies— levy a fixed charge against the underlying property. 
                Generally paid concurrent with the vehicle's registration, the amount of the fee is determined by the property's age and type.
                For example, passenger vehicles face a different fee schedule that motorcycles.
                In all cases, the older a piece of property is, the smaller the associated fee. 
                The full collection of fee schedules can be found on the Tax Commission's website: https://dmv.utah.gov/taxes-fees/uniform-fees 
            """,styles = {'font-size': '14px'})),
    ('Fee-in-lieu Property', pn.pane.Markdown(""" 
                All vehicles not subject to the age-based fee, are instead subject to the fee-in-lieu. 
                This fee is assessed as a percentage (1.5 or 1.0 depending on vehicle type) of the market value of the vehicle.
                This classification covers medium and heavy trucks, commercial trailers, vessels (31 ft or longer), and motor homes.
            """,styles = {'font-size': '14px'})),
    ('Mobile Home', pn.pane.Markdown(""" 
                Mobile or manufactured homes are defined and classified similarly to residential property, including the distinction between primary and non-primary use.
                However, because such homes are ostensibly able to be moved to another site, they are not permanent improvement to the land over the useful life of the property.
                As such, mobile homes are considered personal property rather than real property.
                Even so, the 45% exemption enjoyed by primary residential property would also be enjoyed by a mobile home provided it is used as a primary residence.
                Otherwise, the property is taxed at 100% of market value.
            """,styles = {'font-size': '14px'}))
    ),
                              title = 'Personal Property',
                              collapsed = True,
                              sizing_mode = 'stretch_width')

central_property_primer = pn.Card(
    pn.pane.Markdown(""" 
        Centrally assessed property is primarily distinguished from other types of property since it is the Tax Commission rather than the counties which is responsible for assessment.
        State code lays out the specific types of property that the Commission is directly responsible for including that of public utilities, mobile flight equipment, geothermal resources, mines, or other property which "operates as a unit across county lines" (59-2-201).
        Each property is assessed as a whole before the value is apportioned among the taxing entities in which the property is located.
        For example, a railroad that runs through multiple counties, municipalities, school districts, etc. is assessed on its totality by the Commission, then the Commission determined what part of that total value is taxable by each entity in which the property exists. 
        The taxing entities then levy their rates on the taxable value which has been apportioned to them. 
            """,styles = {'font-size': '16px'}),
                              title = 'Centrally Assessed',
                              collapsed = True,
                              sizing_mode = 'stretch_width')

property_primer = pn.Column('# Property Tax, A Primer',
                            pn.layout.Divider(),
                            pn.pane.Markdown("""
     In Utah, taxable property is divided into one of three broad categories as set out in statute: *real property*, *personal property*, and *centrally assessed property*. 
     Of those, the first two are locally assessed, meaning that property is evaluated at the county level.
     This distinction in the code reflects how the property is used or structured; property that operates as a cohesive whole across multiple jurisdictions is assessed as a cohesive whole.
     A railroad network, for example, does not operate only in a single county and so is assessed at the state level rather than locally. 
     By contrast, something that is fixed in place or registered with a single county (like a home, a business, or a vehicle) is assessed by that county.
     Click on the property types below to learn more.
""",
styles = {'font-size': '20px'}),
pn.layout.Divider(),
pn.layout.Column(real_property_primer, personal_property_primer, central_property_primer),
pn.layout.Divider()
)




In [ ]:
### Truth in Taxation Primer

'''
    Among governmental funding mechansims, property taxes feature prominently.
Like other kinds of taxes, levies on property are generally strucutred as a rate that is charged against the underlying value being taxed.  
In the case of property taxes, that rate is assessed against the value of the property in question.
For example, a local government might set a rate of .001 which would create a tax lability of $400 against a home worth $400,000.
Importantly, the policy strucutre of such taxes is to set the tax rate directly just like is done with a sales tax or an income tax. 
In the case of property however, this structure creates a subtle problem. 
Property, particularly real property, has a habit of appriciating in value.
Mostly, this is a good thing for the owner of the property since they can leverage the appriciated value for whatever purposes they deem worthwhile. 
They might borrow against the higher value, liquidate it by selling the property, or benefit from a number of other more complex financial products.
However, in the event that the owner does not wish to part with their property for whatever reason, such as wanting to continue living in the house that they have owned for decades, a property tax which is strucutred as decribed here creates a situation wherein the tax liabilty continutly increases as the property apprciates in value.
This can be problematic for two reasons.
First, it has the effect raises taxes on a regular basis. 
The reason being that with a fixed rate, tax burden rises with value appriciation. If the home from the previous example were to increase in value to $500,000, the tax on it would rise to $500.
This is different from something like a sales tax where, when the rate is held constant, aggregate liabilty would rise due to an individual choosing to increase consumption.
With property taxes, increasing liability is not a function of consumer choices. 
Second, in general, the pattern of wealth over an individual's life is for it to increase with time until an inflection point where one's wealth begins declining. 
This pattern is driven by individuals on net building wealth over the early and middle decades of life by earning income and accumulating assests, and drawing down that wealth in the decades follwing retirement.
As a consequence, there comes a point in the typical lifetime wealth profile where one's ability to pay increasing property tax liability diminishes at the same time that tax burden continues increasing due to value appriciation.

    Here in Utah however, we do things a little bit differently. 
First enacted in the 1985 session with the Tax Increase Disclosure Act (HB 388), property taxes at all levels are contrained by a process called 'Truth in Taxation.' 
https://images.archives.utah.gov/digital/collection/432n/id/121707/rec/6
It takes the ordinary tax paradigm and flips it on its head.
The basic concept behind Truth in Taxation is to set the revenue to be raised, rather than setting the rate to charge.
More specifically, property tax rates are explicitly strucutred to float up and down in response to changes in the underlying taxable value in order to maintain the same revenue year-to-year. 
At a basic level, this floating action is achived by summing all the taxable property within a district and computing a tax rate which would generate the same revenue as the prior year.
As such, if the aggregate value of property within a district increases, then the applied tax rate would decrease in response and vice versa.

    Truth in Taxation has several obvious advantages over a rate-based system. 
First, residents do not need to bear increases in their tax liabiliy merely becasue their property increased in value. 
Instead, Truth in Taxation aims at keeping one's tax liability constant or near constant year-to-year.
This is not always accomplished perfectly due to complexities in the admisntration and application of the process, but the growth in tax liabilty is systematicly lower than it would be under a rate-based system.
Second, it increases transpanacy and accountability in local taxation.
It does this by requiring that any proposed increase in tax on residents must be explicilty approved by a legislative body with reasonable oppertunity for input from the public.
Under a rate-based system by contrast, increases in tax liabilty would occur near automatically due to increases in property values with no oppertunity for public debate on if the increased revnue was warrented or not.
Third, it provides stability for both taxpayers and local governments. 
For taxpayers, Truth in Taxation ensures that one's tax liability does not swing sharply from year to year merely due to fluctuations or trends in the real estate market. 
As such, they can be assured that their liability will be relativly constant year-to-year, and any substatial increases will come with ample notice.
For local governments, it ensures that they can count on steady revenues when budgeting rather than worrying about the risk of property values changing suddenly in the event of an economic shock.
Consequenlty, local governmetns can have greater confidence when assessing their financial needs and planning how to provide services to their residents.

    Of course, no system is perfect, and Truth in Taxation creates some second order problems.
Fails to adjsut for inflation
The Residential tilt effect

'''


In [ ]:
### Taxable Value Time Series
ratio_timeseries = df_working[df_working.entity_name==entitiy_widget].hvplot.line(
    x = "Tax Year", 
    y = property_share_types, 
    ylabel = 'Taxable Value, Percent',
    title = 'Property Type Shares',
    color = base_colors,
    responsive=True).opts(shared_axes=False)

### County Taxable Value Time Series
value_timeseries = df_working[df_working.entity_name==entitiy_widget].hvplot.line(x = "Tax Year", 
    y = property_types,
    ylabel = 'Taxable Value (B)',
    title = 'Property Type Value',
    color = base_colors,
    responsive=True).opts(shared_axes=False)


In [ ]:
### Average Timeseries
average_timeseries = df_working[df_working.entity_name==entitiy_widget].hvplot.line(
    x = "Tax Year", 
    y = property_average_types, 
    ylabel = 'Average Value',
    title = 'Property Type Averages',
    color = base_colors,
    responsive=True).opts(shared_axes=False)

In [ ]:
### Revenue, Rate Timeseries

revenue_timeseres = df_working[df_working.entity_name == entitiy_widget].hvplot.line(
    x = "Tax Year",
    y = ["Revenue, Total Property"],
    title = 'Entity Revenue',
    color = base_colors,
    responsive = True).opts(shared_axes = False)

rate_timeseres = df_working[df_working.entity_name == entitiy_widget].hvplot.line(
    x = "Tax Year",
    y = ["Real Tax Rate"],
    title = 'Entity Tax Rate',
    color = base_colors,
    responsive = True).opts(shared_axes = False)

In [ ]:
### Histograms, Real tax Rate

taxrate_histogram = df_working[(df_working.entity_type.isin(entity_type_widget)) & (df_working['Tax Year']==tax_year_widget)].hvplot.hist("Real Tax Rate", 
    bins = 50,
    color = highlight,
    ylabel = 'Count',
    xlabel = 'Tax Rate',
    title = "Distribution of Entity Tax Rate",
    hover_cols = [])

property_histogram = df_working[(df_working.entity_type.isin(entity_type_widget)) & (df_working['Tax Year']==tax_year_widget)].hvplot.hist("Total Property", 
    bins = 50,
    color = highlight,
    ylabel = 'Count',
    xlabel = 'Taxable Value (B)',
    title = "Distribution of Property Value",
    hover_cols = [])

realrevenue_histogram = df_working[(df_working.entity_type.isin(entity_type_widget)) & (df_working['Tax Year']==tax_year_widget)].hvplot.hist("Revenue, Total Property", 
    bins = 50,
    color = highlight,
    ylabel = 'Count',
    xlabel = 'Entity Revenue (M)',
    title = "Distribution of Entity Revenue",
    hover_cols = [])

realrevenue_histogram

In [ ]:
### Build App Layout 
# Build Panes
indicators = pn.Column(
    pn.Card(pn.indicators.Number(value = indicator_realrevenue, format = '{value:.3f}'), title = 'Total Revenue (M)'),
    pn.Card(pn.indicators.Number(value = indicator_realproperty, format = '{value:.3f}'), title = 'Total Taxable Property (B)'),
    pn.Card(pn.indicators.Number(value = indicator_realrate, format = '{value:.3f}%'), title = 'Real Tax Rate')
    )

property_type_timeseries = pn.Column(
    pn.Card(pn.pane.HoloViews(ratio_timeseries), title = 'Ratio'), 
    pn.Card(pn.pane.HoloViews(value_timeseries), title = 'Value'))

rate_revenue_timeseres = pn.Column(
    pn.Card(pn.pane.HoloViews(rate_timeseres), title = 'Tax Rate Timeseries'),
    pn.Card(pn.pane.HoloViews(revenue_timeseres), title = 'Revenue Timeseries'))

# Page Switching Function
def show_page(page_key):
    main_area.clear()
    main_area.append(mapping[page_key])

# Initalize Button Connections
frontpage_button.on_click(lambda event: show_page("frontpage"))
entityprofile_button.on_click(lambda event: show_page("entityprofile"))
statewidecomp_button.on_click(lambda event: show_page("statewidecomp"))
modeling_button.on_click(lambda event: show_page("modeling"))

# Create pages
def createpage_frontpage():
    return pn.Column(property_primer)

def createpage_entityprofile():
    return pn.Column(pn.Row(pn.pane.Str(entitiy_widget, styles = {'font-size':'48pt', 'font-weight':' 900'}, align = ('center' ,'center'))),
                     pn.layout.Divider(),
                    pn.Row(pn.Column(entitiy_widget, tax_year_widget, indicators),
                    rate_revenue_timeseres, 
                    property_type_timeseries))

def createpage_statewidecomp():
    return pn.Column(pn.Row(pn.pane.Markdown('# Statewide Comparison')),
    pn.layout.Divider(),
    pn.Row(county_widget, entity_type_widget, tax_year_widget, sort_share_widget, sort_value_widget, ascending_widget),
    pn.Card(pn.pane.HoloViews(ratio_chart), title = 'Ratio'), 
    pn.Card(pn.pane.HoloViews(value_chart), title = "Taxable Value"))

def createpage_modeling():
    return pn.pane.Markdown('# In Progress')

# Set up page mapping
mapping = {
    "frontpage": createpage_frontpage(),
    "entityprofile": createpage_entityprofile(),
    "statewidecomp": createpage_statewidecomp(),
    "modeling": createpage_modeling(),
}

# set up page chunks
main_area = pn.Column(mapping["frontpage"], styles={"width":"100%"})

sidebar_default = pn.Column(pn.pane.PNG("LFA Logo w text.png", width = 250, height = 100, align = ('center', 'center')), 
                    #pn.pane.Markdown("# Pages", align = ('center', 'center')), 
                    frontpage_button, 
                    entityprofile_button, 
                    statewidecomp_button, 
                    modeling_button, 
                    styles={"width": "100%", "padding": "15px"})

# Initalize template
pn.extension(design = 'material')

template = pn.template.MaterialTemplate(
    title = 'Property Tax App',
    header_background = 'Black',
    #logo = "LFA Logo.png",
    sidebar = [sidebar_default],
    sidebar_width = 400,
    main = [main_area]
)

template.show()

2025-08-26 13:03:33,235 ERROR: panel.reactive - Callback failed for object named 'County' changing property {'value': ['Statewide']} 
Traceback (most recent call last):
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\panel\reactive.py", line 464, in _process_events
    self.param.update(**self_params)
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2406, in update
    restore = dict(self_._update(arg, **kwargs))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2439, in _update
    self_._batch_call_watchers()
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2624, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\pa

ERROR:tornado.application:Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x000001AA8F61F830>>, <Task finished name='Task-627' coro=<ServerSession.with_document_locked() done, defined at c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\bokeh\server\session.py:77> exception=IndexError('index 0 is out of bounds for axis 0 with size 0')>)
Traceback (most recent call last):
  File "C:\Users\jgibbs\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 750, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "C:\Users\jgibbs\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 774, in _discard_future_result
    future.result()
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\bokeh\server\session.py", line 98, in _needs_document_lock_wrapper
    result = await result
             ^^^^^^^^^^^^
  F

2025-08-26 13:03:42,981 ERROR: panel.reactive - Callback failed for object named 'Taxing Entity Type' changing property {'value': ['School District']} 
Traceback (most recent call last):
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\panel\reactive.py", line 464, in _process_events
    self.param.update(**self_params)
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2406, in update
    restore = dict(self_._update(arg, **kwargs))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2439, in _update
    self_._batch_call_watchers()
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2624, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site

ERROR:tornado.application:Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x000001AA8F61F830>>, <Task finished name='Task-1094' coro=<ServerSession.with_document_locked() done, defined at c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\bokeh\server\session.py:77> exception=IndexError('index 0 is out of bounds for axis 0 with size 0')>)
Traceback (most recent call last):
  File "C:\Users\jgibbs\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 750, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "C:\Users\jgibbs\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 774, in _discard_future_result
    future.result()
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\bokeh\server\session.py", line 98, in _needs_document_lock_wrapper
    result = await result
             ^^^^^^^^^^^^
  

2025-08-26 13:03:45,657 ERROR: panel.reactive - Callback failed for object named 'Taxing Entity Type' changing property {'value': ['County']} 
Traceback (most recent call last):
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\panel\reactive.py", line 464, in _process_events
    self.param.update(**self_params)
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2406, in update
    restore = dict(self_._update(arg, **kwargs))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2439, in _update
    self_._batch_call_watchers()
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2624, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages

ERROR:tornado.application:Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x000001AA8F61F830>>, <Task finished name='Task-1111' coro=<ServerSession.with_document_locked() done, defined at c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\bokeh\server\session.py:77> exception=IndexError('index 0 is out of bounds for axis 0 with size 0')>)
Traceback (most recent call last):
  File "C:\Users\jgibbs\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 750, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "C:\Users\jgibbs\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 774, in _discard_future_result
    future.result()
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\bokeh\server\session.py", line 98, in _needs_document_lock_wrapper
    result = await result
             ^^^^^^^^^^^^
  

2025-08-26 13:03:50,470 ERROR: panel.reactive - Callback failed for object named 'County' changing property {'value': ['Statewide']} 
Traceback (most recent call last):
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\panel\reactive.py", line 464, in _process_events
    self.param.update(**self_params)
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2406, in update
    restore = dict(self_._update(arg, **kwargs))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2439, in _update
    self_._batch_call_watchers()
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2624, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\pa

ERROR:tornado.application:Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x000001AA8F61F830>>, <Task finished name='Task-1174' coro=<ServerSession.with_document_locked() done, defined at c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\bokeh\server\session.py:77> exception=IndexError('index 0 is out of bounds for axis 0 with size 0')>)
Traceback (most recent call last):
  File "C:\Users\jgibbs\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 750, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "C:\Users\jgibbs\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 774, in _discard_future_result
    future.result()
  File "c:\Users\jgibbs\AppData\Local\Programs\Python\Python312\Lib\site-packages\bokeh\server\session.py", line 98, in _needs_document_lock_wrapper
    result = await result
             ^^^^^^^^^^^^
  